## Imports

In [ ]:
import os
from os.path import join
import sys
from pathlib import Path

# include app directory into sys.path
parent_dir = Path(os.path.abspath('')).parent
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

import torch as pt
from torch.nn.functional import mse_loss
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker 
from flowtorch.analysis import SVD
import numpy as np
from scipy.fft import fft, fftfreq

import utils.config as config
from CNN_VAE.CNN_VAE import make_VAE_model
from FC.FC_model import make_FC_model
from utils.helper_funcs import shift_input_sequence
from utils.DataWindow import DataWindow

plt.rcParams["figure.dpi"] = 180

# use GPU if possible
device = pt.device("cuda") if pt.cuda.is_available() else pt.device("cpu")
print(device)

# define prediction horizon and type of dimensionality reduction
PRED_HORIZON = 4
DIM_REDUCTION = "SVD"       # one of ("SVD" / "VAE")
N_LATENT = config.SVD_rank if DIM_REDUCTION == "SVD" else config.VAE_latent_size
FC_MODEL = config.FC_SVD_single_model if DIM_REDUCTION == "SVD" else config.FC_VAE_single_model

# define paths
DATA_PATH = join(parent_dir, "data", "single_flow_cond")
VAE_PATH = join(parent_dir, "output", "VAE", "latent_study", config.VAE_model)
SVD_PATH = join(parent_dir, "output", "SVD", "U.pt")
FC_PATH = join(parent_dir, "output", "FC", "single", DIM_REDUCTION, "param_study", f"pred_horizon_{PRED_HORIZON}")
OUTPUT_PATH = join(parent_dir, "output", "FC", "single", DIM_REDUCTION, "param_study")

## Pipeline Pre-Processing

In [ ]:
# timestep and index computation, transforming to dimensionsless time
TIMESTEP_1, TIMESTEP_2 = (35, 43)
TIMESTEP_dimless_1= round(((TIMESTEP_1 + (config.single_flow_cond_train_share * config.time_steps_per_cond)) * config.U_inf) / (config.c_mean * config.timesteps_per_second), 2)
TIMESTEP_dimless_2= round(((TIMESTEP_2 + (config.single_flow_cond_train_share * config.time_steps_per_cond))  * config.U_inf) / (config.c_mean * config.timesteps_per_second), 2)

# compute prediction horizons to predict timestep 1 and 2
pred_horizon_1 = TIMESTEP_1 - config.input_width + 1
pred_horizon_2 = TIMESTEP_2 - config.input_width + 1

# set a prediction horizon for comparing latent and full space loss
pred_horizon_total = 32

print(f"Test dataset comprises timesteps {int(config.single_flow_cond_train_share * config.time_steps_per_cond)} - {config.time_steps_per_cond}.")    
print(f"The FC network takes the first {config.input_width} timesteps as input.\n")     
print(f"Predicted timestep 1 (index) is:            {TIMESTEP_1}")
print(f"    which equals a dimensionless time:      {TIMESTEP_dimless_1}")
print(f"    and a prediction horizon of:            {pred_horizon_1}\n")
print(f"Predicted timestep 2 (index) is:            {TIMESTEP_2}")
print(f"    which equals a dimensionless time:      {TIMESTEP_dimless_2}")
print(f"    and a prediction horizon of:            {pred_horizon_2}")

In [ ]:
def reduce_with_VAE(train, test):
    # load pre-trained autoencoder model
    autoencoder = make_VAE_model(
        n_latent=config.VAE_latent_size, 
        device=device)
    autoencoder.load(VAE_PATH)
    autoencoder.eval()
    decoder = autoencoder._decoder

    # reduce datasets
    train_red = autoencoder.encode_dataset(train)
    test_red = autoencoder.encode_dataset(test)

    return train_red, test_red, decoder

def reduce_with_SVD(train, test):
    # load left singular vectors U
    U = pt.load(SVD_PATH)

    # reduce datasets
    train_red = pt.transpose(U[:,:config.SVD_rank], 0, 1) @ (train - train.mean(dim=1).unsqueeze(-1))
    test_red = pt.transpose(U[:,:config.SVD_rank], 0, 1) @ (test - test.mean(dim=1).unsqueeze(-1))

    return train_red, test_red, U[:,:config.SVD_rank]

In [ ]:
# load experimental data
train_data_orig = pt.load(join(DATA_PATH, f"{DIM_REDUCTION}_train.pt"))
test_data_orig = pt.load(join(DATA_PATH, f"{DIM_REDUCTION}_test.pt"))

# load coordinate grids
coords = pt.load(join(Path(DATA_PATH).parent, "coords_interp.pt"))
xx, yy = coords

# load pre-fitted scaler
latent_scaler = pt.load(join(FC_PATH, "scaler.pt"))

In [ ]:
# compress dataset into reduced state either by VAE or SVD
if DIM_REDUCTION == "VAE":
    train_red, test_red, decoder = reduce_with_VAE(train_data_orig, test_data_orig)
elif DIM_REDUCTION == "SVD":
    train_red, test_red, U = reduce_with_SVD(train_data_orig, test_data_orig)
    test_data_orig = test_data_orig.unflatten(dim=0, sizes=config.target_resolution)
else:
    raise ValueError("Unknown DIM_REDUCTION")

In [ ]:
# feed reduced and scaled dataset into DataWindow class to create TimeSeriesTensorDatasets
data_window = DataWindow(train=latent_scaler.scale(train_red), test=latent_scaler.scale(test_red), input_width=config.input_width, pred_horizon=pred_horizon_total)
_, target_idx = data_window.rolling_window(test_red.shape[1])
target_idx = target_idx.tolist()

test_windows = data_window.test_dataset

In [ ]:
# define FC model parameters
_, INPUT_WIDTH, HIDDEN_SIZE, N_HIDDEN_LAYERS = [int(param) for param in FC_MODEL.split("_")]

# create FC model and load model state dict
FC_model = make_FC_model(
    latent_size=N_LATENT,
    input_width=INPUT_WIDTH, 
    hidden_size=HIDDEN_SIZE, 
    n_hidden_layers=N_HIDDEN_LAYERS
)
FC_model.load(join(FC_PATH, FC_MODEL + ".pt"))
FC_model.eval()

# Autoregressive Prediction

In [ ]:
# initialize losses
latent_loss = []
orig_loss = []

with pt.no_grad():
    inputs, targets = test_windows[0]
    # add batch dimension with unsqueeze(0)
    inputs = inputs.flatten().unsqueeze(0).to(device)
    targets = targets.unsqueeze(0).to(device)

    for step in range(pred_horizon_total):
        # shift input sequence by one: add last prediction while discarding first input
        if step != 0:
            inputs = shift_input_sequence(orig_seq=inputs, new_pred=pred)

        # time-evolution (autoregressive)
        pred = FC_model(inputs)
        latent_loss.append(mse_loss(targets[:, :, step], pred))

        # re-scaling
        pred_rescaled = latent_scaler.rescale(pred)

        # expand to full space either by VAE or SVD
        if DIM_REDUCTION == "VAE":
            # forward pass through decoder
            pred_orig = decoder(pred_rescaled.unsqueeze(0)).squeeze().detach() 
        else:
            # matrix multiplication with U, followed by adding back the temporal mean
            pred_orig = (U @ pred_rescaled.permute(1, 0) + test_data_orig.flatten(0, 1).mean(dim=1).unsqueeze(-1)).squeeze().unflatten(dim=0, sizes=config.target_resolution)

        orig_loss.append(mse_loss(test_data_orig[:, :, target_idx[0][step]], pred_orig))

        # if step of specific timestep reached, save to a variable
        if step == pred_horizon_1 - 1:
            pred_1 = pred_orig
        if step == pred_horizon_2 - 1:
            pred_2 = pred_orig

# compute
MSE_1 = (test_data_orig[:, :, TIMESTEP_1] - pred_1)**2
MSE_2 = (test_data_orig[:, :, TIMESTEP_2] - pred_2)**2

#### Plot Latent vs. Full Space Loss

In [ ]:
fig = plt.subplots(1, 1, figsize=config.standard_figsize_1)
plt.plot(range(1, pred_horizon_total + 1), latent_loss, label="reduced space loss")
plt.plot(range(1, pred_horizon_total + 1), orig_loss, label="full space loss")
plt.ylabel("Test MSE")
plt.xlabel("number of autoregressive predictions")
plt.yscale("log")
plt.legend()
plt.tight_layout
plt.savefig(join(OUTPUT_PATH, f"{DIM_REDUCTION}_FC_single_predhor{PRED_HORIZON}_origvslatentloss.png"), bbox_inches="tight")

#### Reconstruct $c_p$-snapshot for timestep 1

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
vmin_cp, vmax_cp = config.plot_lims_cp
vmin_MSE, vmax_MSE = config.plot_lims_MSE_reconstruction
levels_cp = pt.linspace(vmin_cp, vmax_cp, 120)
levels_MSE = pt.linspace(vmin_MSE, vmax_MSE, 120)

ax1.contourf(xx, yy, test_data_orig[:, :, TIMESTEP_1], vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
ax2.contourf(xx, yy, pred_1, vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
cont = ax3.contourf(xx, yy, MSE_1, vmin=vmin_MSE, vmax=vmax_MSE, levels=levels_MSE)

ax1.set_title("Ground Truth")
ax2.set_title(DIM_REDUCTION + "-FC" if DIM_REDUCTION == "SVD" else "CNN-VAE-FC")

fig.subplots_adjust(right=0.95)
cax = fig.add_axes([0.99, 0.283, 0.03, 0.424])
cbar = fig.colorbar(cont, cax=cax,label = "Squarred Error")
cbar.formatter = ticker.FormatStrFormatter(f'%.{3}f')

for ax in [ax1, ax2, ax3]:
    ax.set_aspect("equal")
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])

fig.savefig(join(OUTPUT_PATH, f"{DIM_REDUCTION}_FC_single_predhor{PRED_HORIZON}_timestep_reconstr.png"), bbox_inches="tight")

#### Reconstruct $c_p$-snapshot for timestep 2

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
vmin_cp, vmax_cp = config.plot_lims_cp
vmin_MSE, vmax_MSE = config.plot_lims_MSE_reconstruction
levels_cp = pt.linspace(vmin_cp, vmax_cp, 120)
levels_MSE = pt.linspace(vmin_MSE, vmax_MSE, 120)

ax1.contourf(xx, yy, test_data_orig[:, :, TIMESTEP_2], vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
ax2.contourf(xx, yy, pred_2, vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
cont = ax3.contourf(xx, yy, MSE_2, vmin=vmin_MSE, vmax=vmax_MSE, levels=levels_MSE)

ax1.set_title("Ground Truth")
ax2.set_title(DIM_REDUCTION + "-FC" if DIM_REDUCTION == "SVD" else "CNN-VAE-FC")

fig.subplots_adjust(right=0.95)
cax = fig.add_axes([0.99, 0.283, 0.03, 0.424])
cbar = fig.colorbar(cont, cax=cax,label = "Squarred Error")
cbar.formatter = ticker.FormatStrFormatter(f'%.{3}f')

for ax in [ax1, ax2, ax3]:
    ax.set_aspect("equal")
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])

fig.savefig(join(OUTPUT_PATH, f"{DIM_REDUCTION}_FC_single_predhor{PRED_HORIZON}_timestep_2_reconstr.png"), bbox_inches="tight")

## Reconstruct dataset

In [ ]:
test_reconstr = []
pred_horizon = int(config.time_steps_per_cond - config.single_flow_cond_train_share * config.time_steps_per_cond - config.input_width)

with pt.no_grad():
    inputs, _ = test_windows[0]

    # add batch dimension with unsqueeze(0)
    inputs = inputs.flatten().unsqueeze(0).to(device)

    for step in range(pred_horizon):
        # shift input sequence by one: add last prediction while discarding first input
        if step != 0:
            inputs = shift_input_sequence(orig_seq=inputs, new_pred=pred)

        # time-evolution (autoregressive)
        pred = FC_model(inputs)

        # re-scaling
        pred_rescaled = latent_scaler.rescale(pred)

        # expand to full space either by VAE or SVD
        if DIM_REDUCTION == "VAE":
            # forward pass through decoder
            pred_orig = decoder(pred_rescaled.unsqueeze(0)).squeeze().detach() 
        else:
            # matrix multiplication with U, followed by adding back the temporal mean
            pred_orig = (U @ pred_rescaled.permute(1, 0) + test_data_orig.flatten(0, 1).mean(dim=1).unsqueeze(-1)).squeeze().unflatten(dim=0, sizes=config.target_resolution)

        test_reconstr.append(pred_orig)

test_reconstr = pt.stack(test_reconstr, dim=2)

#### Compare Power Spectra of POD Modes

In [ ]:
# flatten original and reconstructed test dataset
test_original = test_data_orig[:,:,config.input_width:].flatten(0,1)
test_reconstr = test_reconstr.flatten(0,1)

In [ ]:
svd_original= SVD(test_original - test_original.mean(dim=1).unsqueeze(-1), rank=1e5)
V_original = svd_original.V

svd_reconstr = SVD(test_reconstr - test_reconstr.mean(dim=1).unsqueeze(-1), rank=1e5)
V_reconstr = svd_reconstr.V

N = test_original.shape[1]
num_modes = 4
sample_rate = 2000          # [Hz]
y_lims = [1e-7, 1e-1]

fig, ax = plt.subplots(2, 2, figsize=(6, 5), sharex=True)
for row in range(2):
    for col in range(2):
        # Calculate the mode index and retrieve mode coefficients
        mode = row * 2 + col                   
        original_mode_coeffs = V_original[:, mode].numpy()
        reconstr_mode_coeffs = V_reconstr[:, mode].numpy()

        # Compute FFT and PSD
        original_fft = fft(original_mode_coeffs)
        original_psd = np.abs(original_fft)**2 / len(original_fft)
        reconstr_fft = fft(reconstr_mode_coeffs)
        reconstr_psd = np.abs(reconstr_fft)**2 / len(reconstr_fft)

        # Frequency values for plotting
        freq = fftfreq(len(original_mode_coeffs), d=1/sample_rate)* config.U_inf / config.a

        # Use only the positive frequencies (discard negative frequency half)
        freq = freq[:len(freq)//2]
        original_psd = original_psd[:len(original_psd)//2]
        reconstr_psd = reconstr_psd[:len(reconstr_psd)//2]

        # Plot the power spectra
        ax[row, col].semilogy(freq, original_psd, linewidth=0.5, color="black", label="Experimental Data")
        ax[row, col].semilogy(freq, reconstr_psd, linewidth=0.7, color="cornflowerblue", linestyle='dashed', label=DIM_REDUCTION + "-FC" if DIM_REDUCTION == "SVD" else "CNN-VAE-FC")
        ax[row, col].set_title(f"POD Mode {mode + 1}")
        ax[row, col].grid()
        ax[row, col].set_yticklabels([])
        ax[row, col].set_yticks([])
        ax[row, col].set_ylim(y_lims)

        
ax[1, 0].set_xlabel(rf"Reduced Frequency $\omega$")
ax[1, 1].set_xlabel(rf"Reduced Frequency $\omega$")
ax[1,0].legend()

plt.xscale("log")
fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, f"{DIM_REDUCTION}_FC_single_power_spectra.png"), bbox_inches="tight")